In [ ]:
!pip install geopandas
!pip install descartes

     |████████████████████████████████| 972kB 4.3MB/s 
     |████████████████████████████████| 14.8MB 308kB/s 
     |████████████████████████████████| 6.5MB 48.4MB/s 


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
csv_files = ['2000.csv','2001.csv','2002.csv','2003.csv','2004.csv','2005.csv','2006.csv','2007.csv','2008.csv','2009.csv','2010.csv','2011.csv','2012.csv','2013.csv','2014.csv','2015.csv','2016.csv','2017.csv','2018.csv','2019.csv',]
root_dir = '/content/gdrive/My Drive/forecasting'
df_2001 = pd.read_csv(root_dir+'/'+csv_files[1])

In [ ]:
df_2001[df_2001['censuscode']==16]

,DISTRICT,DT_CEN_CD,ST_CEN_CD,ST_NM,censuscode,date,mean
27,Doda,16,1,Jammu & Kashmir,16,2001-01-01,167.311894
653,Doda,16,1,Jammu & Kashmir,16,2001-04-01,185.736818
1307,Doda,16,1,Jammu & Kashmir,16,2001-07-01,308.770629
1904,Doda,16,1,Jammu & Kashmir,16,2001-10-01,168.086426


In [ ]:
"""
datapoints = []

for i in range(641):
    datapoint = []
    for csv_file in csv_files[:-1]:
        csv_file = pd.read_csv(root_dir+'/'+csv_file)
        datapoint = datapoint + csv_file[csv_file['censuscode'] == i].iloc[:]['mean'].to_list()
    if len(datapoint) <11:
        k = sum(datapoint)/len(datapoint)
        l = len(datapoint)
        datapoint = datapoint + [k]*(11-l)
    datapoints.append(datapoint)
labels = []
for i in range(641):
    datapoint = []
    for csv_file in csv_files[3:]:
        csv_file = pd.read_csv(root_dir+'/'+csv_file)
        datapoint =datapoint+ csv_file[csv_file['censuscode'] == i].iloc[:]['mean'].to_list()
    if len(datapoint)<4:
        k = sum(datapoint)/len(datapoint)
        l = len(datapoint)
        datapoint = datapoint+[k]*(4-l)
    assert len(datapoint)==4
    labels.append(datapoint)

"""

In [ ]:
labels=[]
datapoints= []
for r in range(17):
  for i in range(641):
    datapoint = []
    for csv_file in csv_files[r:r+3]:
        csv_file = pd.read_csv(root_dir+'/'+csv_file)
        datapoint = datapoint + csv_file[csv_file['censuscode'] == i].iloc[:]['mean'].to_list()
    if len(datapoint) <12:
        k = sum(datapoint)/len(datapoint)
        l = len(datapoint)
        datapoint = datapoint + [k]*(12-l)
    datapoints.append(datapoint)

  for i in range(641):
    datapoint = []
    for csv_file in {csv_files[r+3]}:
        csv_file = pd.read_csv(root_dir+'/'+csv_file)
        datapoint =datapoint+ csv_file[csv_file['censuscode'] == i].iloc[:]['mean'].to_list()
    if len(datapoint)<4:
        k = sum(datapoint)/len(datapoint)
        l = len(datapoint)
        datapoint = datapoint+[k]*(4-l)
    assert len(datapoint)==4
    labels.append(datapoint)

In [ ]:
datapoints = np.asarray(datapoints)
labels = np.asarray(labels)
print("shape of input : {}".format(datapoints.shape))
print("shape of labels : {}".format(labels.shape))

shape of input : (10897, 12)
shape of labels : (10897, 4)


In [ ]:
#model
input_ = Input((11,1))
layer1 = LSTM(6)(input_)
layer2 = Dense(4,activation='relu')(layer1)
model = Model(input_,layer2)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 11, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 6)                 192       
_________________________________________________________________
dense (Dense)                (None, 4)                 28        
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________


In [ ]:
norm1 = np.linalg.norm(datapoints)
avg_datapoints = datapoints/norm1
avg_labels = labels/np.linalg.norm(labels)
norm2 = np.linalg.norm(labels)
model.compile(Adam(lr=0.001),loss='mse')
history = model.fit(avg_datapoints,avg_labels,epochs=30)

Epoch 1/30
341/341 [==============================] - 2s 5ms/step - loss: 8.1761e-06
Epoch 2/30
341/341 [==============================] - 1s 4ms/step - loss: 7.6586e-06
Epoch 3/30
341/341 [==============================] - 2s 4ms/step - loss: 7.1665e-06
Epoch 4/30
341/341 [==============================] - 1s 4ms/step - loss: 7.1219e-06
Epoch 5/30
341/341 [==============================] - 2s 4ms/step - loss: 7.0755e-06
Epoch 6/30
341/341 [==============================] - 2s 5ms/step - loss: 7.0728e-06
Epoch 7/30
341/341 [==============================] - 1s 4ms/step - loss: 7.0491e-06
Epoch 8/30
341/341 [==============================] - 1s 4ms/step - loss: 7.0409e-06
Epoch 9/30
341/341 [==============================] - 1s 4ms/step - loss: 7.0599e-06
Epoch 10/30
341/341 [==============================] - 1s 4ms/step - loss: 7.0700e-06
Epoch 11/30
341/341 [==============================] - 1s 4ms/step - loss: 7.0572e-06
Epoch 12/30
341/341 [==============================] - 1s 4ms/s

In [ ]:
def get_2004(csv2001,csv2002,csv2003):
    datapoints = []
    print("loading_data...")
    for i in range(641):
        datapoint = []
        for csv_file_ in [csv2001,csv2002,csv2003]:
            csv_file = pd.read_csv(root_dir+'/'+csv_file_)
            datapoint = datapoint + csv_file[csv_file['censuscode'] == i].iloc[:]['mean'].to_list()
        if len(datapoint)!=12:
            k = sum(datapoint)/len(datapoint)
            l = len(datapoint)
            datapoint = datapoint + [k]*(12-l)
        datapoints.append(datapoint)
    print("loading data done...")

    datapoints = np.asarray(datapoints)
    datapoints = datapoints[:,1:]/norm1
    print("predicting...")
    data_2004 = model.predict(datapoints,verbose=1)
    data_2004 = data_2004*norm2
    return data_2004
data = get_2004('2001.csv','2002.csv','2003.csv')

loading_data...
loading data done...
predicting...
21/21 [==============================] - 0s 2ms/step


In [ ]:
data.shape

(641, 4)

In [ ]:
data

array([[  0.      , 352.1465  , 188.65224 ,  31.387178],
       [  0.      , 379.71875 , 281.5581  , 115.326996],
       [  0.      , 387.4791  , 310.81036 , 141.23546 ],
       ...,
       [  0.      , 378.3599  , 263.94196 ,  99.98488 ],
       [  0.      , 391.71228 , 360.4267  , 181.79271 ],
       [  0.      , 396.64255 , 354.92273 , 178.80615 ]], dtype=float32)